# Regression Week 5: LASSO (coordinate descent)


In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

In [ ]:
# Fire up pandas, scikitlearn and numpy
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model

# Load in house sales data

In [ ]:
sales = pd.read_csv('kc_house_data.csv')
train_data = pd.read_csv('kc_house_train_data.csv')
test_data = pd.read_csv('kc_house_test_data.csv')

# Import useful functions from previous notebook

3.. Next, from Module 2 (Multiple Regression), copy and paste the ‘get_numpy_data’ function (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’).

In [152]:
def get_numpy_data(df, features, output):
    df['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of DataFrame given by the features list into the DataFrame features_df (now including constant):
    features_df = df[features]
    #print(df[features])
    # the following line will convert the features_df into a numpy matrix:
    feature_matrix = features_df.values
    # assign the column of data frame associated with the output to the SArray output_sarray
    output_sarray = df[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.values
    return(feature_matrix, output_array)

4.. Similarly, copy and paste the ‘predict_output’ function (or equivalent) from Module 2. This function accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [ ]:
#A function to predict output given regression weights
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

5.. In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

6.. Write a short function called ‘normalize_features(feature_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [ ]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X)

Numpy provides a shorthand for computing 2-norms of each column:

In [ ]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

To normalize, apply element-wise division:

In [ ]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [ ]:
def normalize_features(features):
    #calculate norms
    norms = np.linalg.norm(features, axis=0)
    #calculate normalized features
    normalized_features = features/norms
    return (normalized_features, norms)

To test the function, run the following:

In [ ]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this assignment, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [ ]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [ ]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [ ]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [ ]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [ ]:
#calculate ro[1] approach 1:
i=1
feature_i = simple_feature_matrix[:,i]
w = weights
ro_i = (feature_i *(output - prediction + w[i]*feature_i)).sum()
ro_1

In [ ]:
#calculate ro[1] approach 2:
feature_1 = simple_feature_matrix[:,1]
w = weights
ro_1 = (feature_1 *(output - prediction + w[i]*feature_1)).sum()
ro_1

In [ ]:
2*ro_1

In [ ]:
-2*ro_1

In [ ]:
#calculate ro[2]
feature_2 = simple_feature_matrix[:,2]
w = weights
ro_2 = (feature_2 *(output - prediction + w[2]*feature_2)).sum()
ro_2

In [ ]:
2*ro_2

In [ ]:
# values of l1_penalty would not set w[1] zero: l1_penalty<2*ro_1 or l1_penalty<-2*ro_1

In [ ]:
# values of l1_penalty would set w[2] zero: 11_penalty>=2*ro_2 or l1_penalty >= -2*ro_2

#the combined range of l1_penalty: [2*ro_2 ,2*ro_1]

Answer: [161933397.33247894, 175878941.64650351]



***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [ ]:
# values of l1_penalty would set w[1] zero: l1_penalty>=2*ro_1 or l1_penalty >= -2*ro_2
# values of l1_penalty would set w[2] zero: l1_penalty>=2*ro_2 or l1_penalty >= -2*ro_2

#the combined range of l1_penalty: l1_penalty>2*ro_1

Answer: l1_penalty>175878941.64650351

So we can say that ro[i] quantifies the significance of the i-th feature: the larger ro[i] is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [ ]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] *(output - prediction + weights[i]*feature_matrix[:,i])).sum()
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

If you are using Numpy, test your function with the following snippet:

In [126]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [127]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] *(output - prediction + weights[i]*feature_matrix[:,i])).sum()
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

In [168]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # while we haven't reached the tolerance yet, update each feature's weight
        changes = []
        for i in range(len(weights)):
            #print('i is:', i)
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            #print('old_weights_i', old_weights_i)
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            #print('new_weights_i', weights[i])
            # use old_weights_i to compute change in coordinate
            changes_i = np.abs(weights[i]-old_weights_i)
            #print('changes_i', changes_i)
            changes.append(changes_i) 
        if max(changes)<tolerance:
            #print('converged')
            converged = True
        else:
            converged = False
    return weights
    

Using the following parameters, learn the weights on the sales dataset. 

In [169]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [170]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [171]:
new_weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,initial_weights, l1_penalty, tolerance)

In [172]:
new_weights

array([ 21624997.95951909,  63157247.20788956,         0.        ])

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [173]:
prediction = predict_output(normalized_simple_feature_matrix, new_weights)
error = prediction - output
RSS = np.sum(error*error)
print ('Answer: RSS is', RSS)

Answer: RSS is 1.63049247672e+15


Answer: bedroom has a weight of 0

# Evaluating LASSO fit with more features

17..Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call ‘random_split’ with .8 ratio and seed=0. Otherwise, please down the corresponding csv files from the downloads section.

18..Create a normalized feature matrix from the TRAINING data with the following set of features.

~bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

Make sure you store the norms for the normalization, since we’ll use them later.

In [174]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
my_output = 'price'

In [175]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix) # normalize features

19.. First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [176]:
initial_weights = np.zeros(14)
l1_penalty = 1e7
tolerance = 1.0

In [177]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,initial_weights, l1_penalty, tolerance)


In [178]:
weights1e7

array([ 24429600.23440312,         0.        ,         0.        ,
        48389174.77154896,         0.        ,         0.        ,
         3317511.21492165,   7329961.81171425,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

20.. Quiz Question: What features had non-zero weight in this case?

Answer: intercept, sqft_living, waterfront, view

21.. Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights ‘weights1e8’, you will need them later.

In [179]:
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,initial_weights, l1_penalty, tolerance)

In [180]:
weights1e8

array([ 71114625.71488702,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

22.. Quiz Question: What features had non-zero weight in this case?

Answer: intercept

23.. Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights ‘weights1e4’, you will need them later. (This case will take quite a bit longer to converge than the others above.)

24.. Quiz Question: What features had non-zero weight in this case?

In [181]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,initial_weights, l1_penalty, tolerance)

In [182]:
weights1e4

array([ 78564738.34156762, -22097398.92430532,  12791071.87278517,
        93808088.09281193,  -2013172.75704954,  -4219184.93265014,
         6482842.81753506,   7127408.53480689,   5001664.8546964 ,
        14327518.43714051, -15770959.15237397,  -5159591.22213147,
       -84495341.7684364 ,   2824439.49703683])

Answer: all features have non-zeros

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [185]:
normalized_weights1e4 = weights1e4/all_norms
normalized_weights1e7 = weights1e7/all_norms
normalized_weights1e8 = weights1e8/all_norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [186]:
print (normalized_weights1e7[3])

161.317457646


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [187]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [188]:
prediction = predict_output(normalized_all_feature_matrix, normalized_weights1e4)
error = prediction - output
RSS = np.sum(error*error)
print ('Answer: RSS of weightsle4 is', RSS)

Answer: RSS of weightsle4 is 7.32167050665e+15


In [189]:
prediction = predict_output(normalized_all_feature_matrix, normalized_weights1e7)
error = prediction - output
RSS = np.sum(error*error)
print ('Answer: RSS of weightsle7 is', RSS)

Answer: RSS of weightsle7 is 7.39023167184e+15


In [190]:
prediction = predict_output(normalized_all_feature_matrix, normalized_weights1e8)
error = prediction - output
RSS = np.sum(error*error)
print ('Answer: RSS of weightsle8 is', RSS)

Answer: RSS of weightsle8 is 7.35662905696e+15


***QUIZ QUESTION***

Which model performed best on the test data?

Answer: normalized_weights1e4